# Aula 2: Construindo APIs com Flask

Nesta aula aprofundaremos:

- O que é Flask e por que escolher este microframework
- Instalação segura em ambiente virtual e estrutura de projeto avançada
- Definição de rotas, métodos HTTP e o objeto `request`
- Retornando respostas JSON: `jsonify` vs `json.dumps`
- Estrutura de uma API CRUD completa com explicações passo a passo
- Tratamento de erros com `abort` e handlers personalizados
- Organização de código com Blueprints para escalabilidade
- Testes de endpoints com `curl`, HTTPie e Postman


## 1. O que é Flask e por que usá-lo

Flask é um **microframework** que implementa a especificação WSGI, fornecendo apenas o núcleo mínimo:

- **Leve**: não inclui camadas desnecessárias como ORM ou sistema de formulários por padrão.
- **Flexível**: você escolhe bibliotecas adicionais (SQLAlchemy, Marshmallow, Flask-RESTful etc.).
- **Rápido para prototipação**: ideal para APIs que não requerem toda a complexidade de frameworks full-stack.

> **Observação:** ao contrário de frameworks “monolíticos” (Django), Flask deixa claro quais componentes você precisa, evitando dependências desnecessárias.

## 2. Instalação e ambiente virtual

1. **Criar ambiente virtual** (isolamento de dependências):

   ```bash
   python3 -m venv venv           # Cria o venv
   source venv/bin/activate       # Ativa no Linux/Mac
   venv\Scripts\activate        # Ativa no Windows
   ```

2. **Instalar Flask**:

   ```bash
   pip install Flask
   ```

3. **Fixar versões**:

   ```bash
   pip freeze > requirements.txt
   ```

4. **Estrutura de projeto recomendada**:

```
api_flask/
├── app.py                # Ponto de entrada da aplicação
├── requirements.txt      # Dependências
├── config.py             # Configurações (desenvolvimento, produção)
└── blueprints/
    ├── __init__.py       # Inicializa pacotes
    └── items.py          # Rotas dos recursos 'items'
```

## 3. Aplicação Flask mínima explicada

```python
# app.py
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello, Flask!'

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
```

- `Flask(__name__)`: define o pacote/app; `__name__` ajuda o Flask a localizar recursos.
- `@app.route('/')`: decorator que registra a rota `/` para o método `index()`.
- `debug=True`: habilita recarregamento automático e debugger interativo (não use em produção).
- `host='0.0.0.0'`: torna o serviço acessível externamente; padrão é `127.0.0.1`.
- `port=5000`: porta HTTP padrão do Flask.

## 4. Rotas, métodos HTTP e o objeto `request`

```python
from flask import request

@app.route('/items', methods=['GET', 'POST'])
def handle_items():
    if request.method == 'GET':
        # request.args: parâmetros de query string
    elif request.method == 'POST':
        # request.get_json(): corpo JSON
```

- `methods`: lista de verbos HTTP aceitos.
- `request.args`: `ImmutableMultiDict` com parâmetros de URL.
- `request.form`: dados de formulário `x-www-form-urlencoded`.
- `request.get_json()`: analisa JSON do corpo (retorna dict ou None).
- `request.headers`: dict dos cabeçalhos HTTP.

## 5. Retornando JSON: `jsonify` vs `json.dumps`

- **`jsonify`**:
  - Define automaticamente `Content-Type: application/json`
  - Serializa objetos Python (listas, dicts)
  - Usa `flask.json.provider` para suporte customizado

```python
from flask import jsonify

@app.route('/ping')
def ping():
    data = {'message': 'pong'}
    return jsonify(data), 200
```

- **`json.dumps`**:
  - Retorna string JSON, mas não ajusta headers

```python
import json
from flask import Response

@app.route('/ping2')
def ping2():
    json_str = json.dumps({'message': 'pong'})
    return Response(json_str, mimetype='application/json'), 200
```

## 6. Exemplo Prático: API CRUD com passo a passo

Vamos construir uma API para gerenciar **itens** em memória, comentando cada parte.


In [ ]:
# app.py
from flask import Flask, request, jsonify, abort

app = Flask(__name__)

# 1. Banco de dados em memória (lista de dicts)
items = [
    {'id': 1, 'name': 'Apple', 'price': 0.5},
    {'id': 2, 'name': 'Banana', 'price': 0.3}
]

def find_item(item_id):
    """Busca um item pelo ID, retornando None se não existir."""
    return next((item for item in items if item['id'] == item_id), None)

# 2. GET /items: retorna toda a lista
@app.route('/items', methods=['GET'])
def get_items():
    return jsonify(items), 200

# 3. GET /items/<id>: retorna um único item
@app.route('/items/&lt;int:item_id&gt;', methods=['GET'])
def get_item(item_id):
    item = find_item(item_id)
    if not item:
        abort(404, description='Item não encontrado')
    return jsonify(item), 200

# 4. POST /items: cria um novo item
@app.route('/items', methods=['POST'])
def create_item():
    data = request.get_json()
    # Validação básica
    if not data or 'name' not in data or 'price' not in data:
        abort(400, description='Dados inválidos')
    new_id = max(item['id'] for item in items) + 1
    item = {'id': new_id, 'name': data['name'], 'price': data['price']}
    items.append(item)
    return jsonify(item), 201

# 5. PUT /items/<id>: atualiza um item existente
@app.route('/items/&lt;int:item_id&gt;', methods=['PUT'])
def update_item(item_id):
    item = find_item(item_id)
    if not item:
        abort(404, description='Item não encontrado')
    data = request.get_json()
    # Atualiza somente campos informados
    for key in ('name', 'price'):
        if key in data:
            item[key] = data[key]
    return jsonify(item), 200

# 6. DELETE /items/<id>: remove um item
@app.route('/items/&lt;int:item_id&gt;', methods=['DELETE'])
def delete_item(item_id):
    item = find_item(item_id)
    if not item:
        abort(404, description='Item não encontrado')
    items.remove(item)
    return '', 204

# 7. Error handlers personalizados
@app.errorhandler(404)
def not_found(e):
    return jsonify({'error': str(e)}), 404

@app.errorhandler(400)
def bad_request(e):
    return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True)


## 7. Tratamento de erros avançado

- `abort(code, description)`: interrompe a execução e retorna erro.
- `@app.errorhandler(code)`: personaliza resposta de erro.

```python
@app.errorhandler(500)
def internal_error(e):
    return jsonify({'error': 'Erro interno'}), 500
```

## 8. Organização com Blueprints

**Vantagens:** modularidade, separação de responsabilidades e facilidade de testes.

```bash
api_flask/
├── app.py
├── blueprints/
│   ├── __init__.py
│   └── items.py
└── config.py
```

**Exemplo `blueprints/items.py`:**
```python
from flask import Blueprint, request, jsonify, abort

bp = Blueprint('items', __name__, url_prefix='/items')

@bp.route('', methods=['GET'])
def list_items():
    # ...

@bp.route('', methods=['POST'])
def add_item():
    # ...
``` 

No `app.py`:
```python
from flask import Flask
from blueprints.items import bp as items_bp

app = Flask(__name__)
app.register_blueprint(items_bp)
```

## 9. Testando endpoints

- **curl**:

```bash
curl -i http://localhost:5000/items
curl -X POST -H 'Content-Type: application/json' -d '{"name":"Orange","price":0.4}' http://localhost:5000/items
```

- **HTTPie** (mais legível):

```bash
http GET localhost:5000/items
http POST localhost:5000/items name=Orange price:=0.4
```

- **Postman**:
  1. Crie coleção e requisição.
  2. Selecione método e URL.
  3. No body, escolha `raw` + `JSON`, cole payload.
  4. Envie e analise resposta e headers.

Pronto! Sua API Flask está bem estruturada e documentada.